#  "Explorando el corazón: Utilizando datos clínicos y de estilo de vida para predecir enfermedades cardíacas"

#### ENTRENAMIENTO DE LOS MODELOS

A lo largo de este notebook, abordaremos el problema de clasificación y nos enfocaremos en entrenar varios modelos con el objetivo de obtener las métricas más óptimas para nuestro problema. Nuestro objetivo final es encontrar el modelo que brinde el mejor rendimiento y precisión en la clasificación de los datos. A través de este proceso de entrenamiento y evaluación, exploraremos diferentes algoritmos y técnicas para optimizar nuestros modelos y lograr resultados confiables y eficientes. Estaremos atentos a métricas como la precisión, la exactitud, el puntaje F1 y otras medidas relevantes para evaluar el desempeño de cada modelo. Al final de este notebook, esperamos haber identificado el modelo más adecuado que se ajuste a nuestras necesidades y nos proporcione las mejores métricas en nuestra tarea de clasificación.
1. Logistic Regresion
2. 
3. 
4. 
5. 
6. 

In [17]:
# Importamos las distintas librerias necesarias para el análisis

# Tratamiento de datos
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,log_loss, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV


# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
# Leemos el archivo de data/preprocessed_heart
df= pd.read_csv("../data/processed/processed_heart.csv")
df.head()

,HeartDisease,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,PhysicalActivity,Asthma,KidneyDisease,SkinCancer,AgeCategory_encoded,Race_encoded,Diabetic_encoded,GenHealth_encoded,BMI_Category_Ordinal,GrupoSalud_Ordinal,GrupoSalud_Mental_Ordinal,SleepGroup_Ordinal
0,0,1,0,0,0,0,1,1,0,1,7,5,2,4,1,1,3,1
1,0,0,0,1,0,0,1,0,0,0,12,5,0,4,2,1,1,3
2,0,1,0,0,0,1,1,1,0,0,9,5,2,1,3,3,3,3
3,0,0,0,0,0,0,0,0,0,1,11,5,0,2,2,1,1,2
4,0,0,0,0,1,0,1,0,0,0,4,5,0,4,2,3,1,3


In [19]:
# Para observar mejor la correlación que se produce con respecto a la varible target de "HeartDisease"
# Se muestra en porcentage % y por orden ascendente
corr_matrix = df.corr()
corr_matrix['HeartDisease'].sort_values(ascending=False) *100

HeartDisease                 100.000000
AgeCategory_encoded           23.343224
DiffWalking                   20.125805
Stroke                        19.683530
Diabetic_encoded              16.855285
GrupoSalud_Ordinal            16.731876
KidneyDisease                 14.519710
Smoking                       10.776416
SkinCancer                     9.331688
Sex                            7.004048
BMI_Category_Ordinal           5.342468
Asthma                         4.144415
Race_encoded                   3.485362
GrupoSalud_Mental_Ordinal      2.130066
GenHealth_encoded             -1.106186
AlcoholDrinking               -3.207974
SleepGroup_Ordinal            -3.456049
PhysicalActivity             -10.002993
Name: HeartDisease, dtype: float64

In [20]:
# Usamos skew para calcular la asimetría de nuestro DataFrame
    # Un valor positivo indica una cola más larga hacia la derecha, lo que significa que hay valores extremos más altos. 
    # Un valor negativo indica una cola más larga hacia la izquierda, lo que significa que hay valores extremos más bajos.
df.skew(axis=0)

HeartDisease                 2.962525
Smoking                      0.355585
AlcoholDrinking              3.429019
Stroke                       4.851460
DiffWalking                  2.088606
Sex                          0.099029
PhysicalActivity            -1.319602
Asthma                       2.148059
KidneyDisease                4.918135
SkinCancer                   2.797757
AgeCategory_encoded         -0.263611
Race_encoded                -1.923663
Diabetic_encoded             2.088800
GenHealth_encoded           -0.129635
BMI_Category_Ordinal         0.659454
GrupoSalud_Ordinal           2.199359
GrupoSalud_Mental_Ordinal    1.808620
SleepGroup_Ordinal          -1.391516
dtype: float64

In [21]:
# Hacemos un bucle para comprobar todas las columnas cuales están desbalanceadas y cuales no
for column in df.columns:
    print(df[column].value_counts(sort=True))  # Contamos los valores de la columna y mostramos los resultados ordenados
    print()

HeartDisease
0    292422
1     27373
Name: count, dtype: int64

Smoking
0    187887
1    131908
Name: count, dtype: int64

AlcoholDrinking
0    298018
1     21777
Name: count, dtype: int64

Stroke
0    307726
1     12069
Name: count, dtype: int64

DiffWalking
0    275385
1     44410
Name: count, dtype: int64

Sex
0    167805
1    151990
Name: count, dtype: int64

PhysicalActivity
1    247957
0     71838
Name: count, dtype: int64

Asthma
0    276923
1     42872
Name: count, dtype: int64

KidneyDisease
0    308016
1     11779
Name: count, dtype: int64

SkinCancer
0    289976
1     29819
Name: count, dtype: int64

AgeCategory_encoded
9     34151
8     33686
10    31065
7     29757
6     25382
12    24153
5     21791
11    21482
0     21064
4     21006
3     20550
2     18753
1     16955
Name: count, dtype: int64

Race_encoded
5    245212
3     27446
2     22939
4     10928
1      8068
0      5202
Name: count, dtype: int64

Diabetic_encoded
0    269653
2     40802
1      6781
3      2559
N

# Probamos el modelo sin aplicar el desbalanceo

In [22]:
df = df.drop(columns=(["PhysicalActivity", "SleepGroup_Ordinal","AlcoholDrinking", "GenHealth_encoded", "GrupoSalud_Mental_Ordinal",
                       "Race_encoded","Asthma", "BMI_Category_Ordinal", "SkinCancer", "KidneyDisease"]))

In [23]:
#definimos nuestras etiquetas y features
y = df['HeartDisease']
X = df.drop('HeartDisease', axis=1)

In [24]:
#dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [25]:
# Escalar las características 

scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) # solo transform no fit(), ya que sino contaminariamos los datos 

In [26]:
logre = LogisticRegression(C=1.0,penalty='l2',random_state=42,solver="newton-cg")
logre.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42, solver='newton-cg')

In [27]:
y_pred = logre.predict(X_test_scaled) 


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Suponiendo que ya has realizado todas las operaciones previas y tienes las variables adecuadas

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

# Calcular la precisión por clase
precision = precision_score(y_test, y_pred)
print("Precisión por clase:", precision)

# Calcular el recall (sensibilidad) por clase
recall = recall_score(y_test, y_pred)
print("Recall por clase:", recall)

# Calcular la puntuación F1 por clase
f1 = f1_score(y_test, y_pred)
print("Puntuación F1 por clase:", f1)

# Calcular el classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Precisión del modelo: 0.9125689895089042
Precisión por clase: 0.5
Recall por clase: 0.08369098712446352
Puntuación F1 por clase: 0.14338235294117646
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     58367
           1       0.50      0.08      0.14      5592

    accuracy                           0.91     63959
   macro avg       0.71      0.54      0.55     63959
weighted avg       0.88      0.91      0.88     63959



In [29]:
X.columns

Index(['Smoking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory_encoded',
       'Diabetic_encoded', 'GrupoSalud_Ordinal'],
      dtype='object')

In [30]:
corr_matrix = df.corr()
corr_matrix['HeartDisease'].sort_values(ascending=False) *100

HeartDisease           100.000000
AgeCategory_encoded     23.343224
DiffWalking             20.125805
Stroke                  19.683530
Diabetic_encoded        16.855285
GrupoSalud_Ordinal      16.731876
Smoking                 10.776416
Sex                      7.004048
Name: HeartDisease, dtype: float64

**EXPLICACIÓN DE LAS MÉTRICAS**

Precisión del modelo: 0.9137785467849363. Esta métrica representa la proporción de predicciones correctas sobre el total de predicciones realizadas. En este caso, el modelo tiene una precisión del 91.38%, lo que significa que aproximadamente el 91.38% de las predicciones son correctas.

Precisión por clase: 0.5062068965517241. Esta métrica representa la proporción de predicciones positivas que son correctas para la clase "1" (enfermedad cardíaca) en relación con todas las predicciones positivas realizadas para esa clase. Aquí, la precisión para la clase "1" es del 50.62%, lo que indica que alrededor del 50.62% de las predicciones positivas para la enfermedad cardíaca son correctas.

Recall por clase: 0.08854041013268998. Esta métrica, también conocida como sensibilidad o tasa de verdaderos positivos, representa la proporción de casos positivos (enfermedad cardíaca) que son correctamente identificados por el modelo. En este caso, el recall para la clase "1" es del 8.85%, lo que significa que el modelo identifica correctamente aproximadamente el 8.85% de los casos reales de enfermedad cardíaca.

Puntuación F1 por clase: 0.15071868583162218. La puntuación F1 es una medida que combina la precisión y el recall en una única métrica. Es útil cuando hay un desequilibrio entre las clases, como en este caso. La puntuación F1 para la clase "1" es del 15.07%, lo que indica un equilibrio entre la precisión y el recall para esa clase.

En cuanto al classification report, proporciona un resumen detallado de las métricas por clase y para el conjunto de datos completo. Observamos que la clase "0" (no enfermedad cardíaca) tiene una precisión alta, recall alto y puntuación F1 alta, mientras que la clase "1" (enfermedad cardíaca) muestra valores más bajos en todas las métricas. Esto puede indicar un desequilibrio en los datos, donde la clase "1" tiene menos representación en comparación con la clase "0". El conjunto de datos está sesgado hacia la clase "0" y eso puede afectar el rendimiento del modelo en la detección de la clase minoritaria (enfermedad cardíaca).

En resumen, las métricas indican que el modelo tiene una precisión general alta, pero tiene dificultades para identificar correctamente los casos de enfermedad cardíaca. Esto puede deberse al desequilibrio en los datos y es importante tenerlo en cuenta al interpretar los resultados y tomar decisiones basadas en ellos.

In [31]:
# Calcular la matriz de confusión 

confusion = confusion_matrix(y_test, y_pred) 
print("Matriz de confusión:") 
print(confusion) 

Matriz de confusión:
[[57899   468]
 [ 5124   468]]


In [32]:
# Calcular el classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     58367
           1       0.50      0.08      0.14      5592

    accuracy                           0.91     63959
   macro avg       0.71      0.54      0.55     63959
weighted avg       0.88      0.91      0.88     63959



## APLICAREMOS LAS DISTINTAS TÉCNICAS PARA DESBALANCEAR EL PROBLEMA

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
 
from pylab import rcParams
 
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
 
from collections import Counter

En este apartado aplicaremos diferentes técnicas de desbalanceo de los datos aplicados a un mismo modelo para poder ver cual es la más acertada y así quedarnos con una...............

### Estrategia: Penalización para compensar

Utilizaremos un parámetro adicional en el modelo de Regresión logística en donde indicamos weight = “balanced” y con esto el algoritmo se encargará de equilibrar a la clase minoritaria durante el entrenamiento

In [59]:
#dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# EscalaMOS las características 
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) # solo transform no fit(), ya que sino contaminariamos los datos 

# Modelo de Logistic Regression
logre2 = LogisticRegression(C=0.1,penalty='l1',random_state=42,solver="liblinear", class_weight= "balanced") # únicamente incorporamos el class_weight= "balanced"
logre2.fit(X_train_scaled, y_train)
y_pred2 = logre2.predict(X_test_scaled) 

In [60]:
# Calcular el classification report
report2 = classification_report(y_test, y_pred2)
print("Classification Report:")
print(report2)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.73      0.83     58367
           1       0.21      0.77      0.33      5592

    accuracy                           0.73     63959
   macro avg       0.59      0.75      0.58     63959
weighted avg       0.90      0.73      0.79     63959



**EXPLICACIÓN**

En comparación con el modelo anterior, el nuevo modelo de Logistic Regression con los datos de prueba actualizados muestra una mejora en ciertas métricas, mientras que otras disminuyen. Aquí hay un resumen de las diferencias clave:

Precisión general: El nuevo modelo tiene una precisión general de 0.74, lo que indica que aproximadamente el 74% de las predicciones son correctas. Esta precisión es menor que la del modelo anterior, que era de 0.91.

Precisión por clase: Para la clase "0" (no enfermedad cardíaca), la precisión es alta con un valor de 0.97, lo que indica que la mayoría de las predicciones negativas son correctas. Sin embargo, para la clase "1" (enfermedad cardíaca), la precisión es baja con un valor de 0.22, lo que indica que la mayoría de las predicciones positivas son incorrectas.

Recall por clase: El nuevo modelo muestra un recall alto para la clase "1" (enfermedad cardíaca) con un valor de 0.77, lo que indica que la mayoría de los casos reales de enfermedad cardíaca son correctamente identificados por el modelo. Sin embargo, el recall para la clase "0" (no enfermedad cardíaca) es de 0.74, lo que indica que hay algunos casos reales de no enfermedad cardíaca que no son identificados por el modelo.

Puntuación F1 por clase: La puntuación F1 para la clase "1" (enfermedad cardíaca) es de 0.34, lo que indica un equilibrio entre la precisión y el recall para esa clase. Sin embargo, la puntuación F1 para la clase "0" (no enfermedad cardíaca) es de 0.84, lo que indica un mejor equilibrio entre precisión y recall para esa clase.

En general, el nuevo modelo muestra una mejora en la identificación de casos de enfermedad cardíaca, como se refleja en un recall más alto para la clase "1". Sin embargo, la precisión para la clase "1" ha disminuido, lo que significa que hay un mayor número de falsos positivos en comparación con el modelo anterior. Además, la precisión general y la puntuación F1 han disminuido. Estos resultados sugieren que el modelo todavía puede beneficiarse de ajustes o considerar técnicas adicionales para abordar el desequilibrio en los datos y mejorar la detección de enfermedades cardíacas.

In [36]:
# Calculamos la matriz de confusión
confusion2 = confusion_matrix(y_test, y_pred2)
print("Matriz de confusión:")
print(confusion2)




Matriz de confusión:
[[42480 15887]
 [ 1280  4312]]


### Estrategia: Oversampling de la clase minoritaria

En este caso, crearemos muestras nuevas “sintéticas” de la clase minoritaria. Usando RandomOverSampler.

In [57]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from collections import Counter

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar RandomOverSampler
os = RandomOverSampler(sampling_strategy=0.5)
X_train_res, y_train_res = os.fit_resample(X_train_scaled, y_train)

# Imprimir distribución antes y después del resampling
print("Distribución antes del resampling:", Counter(y_train))
print("Distribución después del resampling:", Counter(y_train_res))

# Definir y entrenar el modelo
model = LogisticRegression(C=0.1,penalty='l2',random_state=42,solver="newton-cg", class_weight= "balanced") # únicamente incorporamos el class_weight= "balanced"
model.fit(X_train_res, y_train_res)

# Realizar predicciones en el conjunto de prueba
pred_y_random = model.predict(X_test_scaled)


Distribución antes del resampling: Counter({0: 234055, 1: 21781})
Distribución después del resampling: Counter({0: 234055, 1: 117027})


In [58]:
# Calcular el classification report
report4 = classification_report(y_test, pred_y_random)
print("Classification Report:")
print(report4)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.73      0.83     58367
           1       0.21      0.77      0.33      5592

    accuracy                           0.73     63959
   macro avg       0.59      0.75      0.58     63959
weighted avg       0.90      0.73      0.79     63959



In [236]:
# Calculamos la matriz de confusión
confusion4 = confusion_matrix(y_test, pred_y_random)
print("Matriz de confusión:")
print(confusion4)

Matriz de confusión:
[[42593 15774]
 [ 1302  4290]]


### Estrategia: Combinamos resampling con Smote-Tomek

Ahora probaremos una técnica muy usada que consiste en aplicar en simultáneo un algoritmo de subsampling y otro de oversampling a la vez al dataset. En este caso usaremos SMOTE para oversampling: busca puntos vecinos cercanos y agrega puntos “en linea recta” entre ellos. Y usaremos Tomek para undersampling que quita los de distinta clase que sean nearest neighbor y deja ver mejor el decisión boundary (la zona limítrofe de nuestras clases).

In [29]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from collections import Counter

# Aplicar SMOTE para oversampling
smote = SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Aplicar Tomek Links para undersampling
tomek = TomekLinks()
X_train_res, y_train_res = tomek.fit_resample(X_train_res, y_train_res)

print("Distribución antes del resampling:", Counter(y_train))
print("Distribución después del resampling:", Counter(y_train_res))

# Entrenar un modelo (reemplaza esta parte con tu propio código de entrenamiento de modelo)
model = LogisticRegression(C=1.0, penalty='l2', random_state=42, solver="newton-cg", class_weight="balanced")
model.fit(X_train_res, y_train_res)

# Realizar predicciones en el conjunto de prueba
pred_y = model.predict(X_test)





Distribución antes del resampling: Counter({0: 234055, 1: 21781})
Distribución después del resampling: Counter({0: 234055, 1: 233355})


In [30]:
# Calcular el classification report
report3 = classification_report(y_test, pred_y)
print("Classification Report:")
print(report3)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.70      0.81     58367
           1       0.20      0.78      0.31      5592

    accuracy                           0.70     63959
   macro avg       0.58      0.74      0.56     63959
weighted avg       0.90      0.70      0.77     63959



In [31]:
# Calculamos la matriz de confusión
confusion3 = confusion_matrix(y_test, pred_y)
print("Matriz de confusión:")
print(confusion3)

Matriz de confusión:
[[40667 17700]
 [ 1252  4340]]


### Estrategia: Ensamble de Modelos con Balanceo

Para esta estrategia usaremos un Clasificador de Ensamble que utiliza Bagging y el modelo será un DecisionTree

In [42]:
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=True,
                                random_state=0,)
 
#Train the classifier.
bbc.fit(X_train, y_train)
pred_yBagging = bbc.predict(X_test)


In [43]:
# Calcular el classification report
report4 = classification_report(y_test, pred_yBagging)
print("Classification Report:")
print(report4)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.70      0.81     58367
           1       0.20      0.80      0.32      5592

    accuracy                           0.71     63959
   macro avg       0.59      0.75      0.57     63959
weighted avg       0.91      0.71      0.77     63959



In [44]:
# Calculamos la matriz de confusión
confusion4 = confusion_matrix(y_test, pred_yBagging)
print("Matriz de confusión:")
print(confusion4)

Matriz de confusión:
[[40827 17540]
 [ 1118  4474]]


### Resultados de las Estrategias


### MODELOS A TRATAR

A lo largo de este apartado realizaremos la solución de problema a través de diferentes modelos supervisados entre los que nos encontramos con..... en busca de las mejores métricas posibles para nuestro problema.......

#### LOGISTIC REGRESSION

In [6]:
# Escalamos los datos
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X)


In [7]:
# El uso de stratify = y es clave para que dicha variable tenga datos balanceados
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0, stratify=y)

print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)



X_train (255836, 7)
X_test (63959, 7)
y_train (255836,)
y_test (63959,)


In [8]:
# Escalamos los datos de X_test, pero no se entrenan
X_test_scaled = scaler.transform(X_test)

In [12]:
# Modelos que quiero probar
classifiers = [
    KNeighborsClassifier(2),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GaussianNB(),
    LogisticRegression(),
    XGBClassifier()]

In [13]:
for classifier in classifiers:
    # Entrenar el clasificador
    classifier.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = classifier.predict(X_test)

    # Calcular la precisión
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Metrics ({classifier.__class__.__name__}):")
    print(f"Accuracy: {accuracy:.5f}")
    print(f"Precision: {precision:.5f}")
    print(f"Recall: {recall:.5f}")
    print(f"F1 Score: {f1:.5f}")

Metrics (KNeighborsClassifier):
Accuracy: 0.90874
Precision: 0.32461
Recall: 0.06119
F1 Score: 0.10297


### MODELOS SUPERVISADOS

**¿Qué es el aprendizaje supervisado?**


#### LOGISTIC REGRESION

In [ ]:
#dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# EscalaMOS las características 
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) # solo transform no fit(), ya que sino contaminariamos los datos 

# Modelo de Logistic Regression
logre2 = LogisticRegression(C=1.0,penalty='l2',random_state=42,solver="newton-cg", class_weight= "balanced") # únicamente incorporamos el class_weight= "balanced"
logre2.fit(X_train_scaled, y_train)
y_pred2 = logre2.predict(X_test_scaled) 

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest


In [46]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Crear el pipeline con escalamiento y modelo Logistic Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selectkbest', SelectKBest()),
    ('logre2', LogisticRegression())
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'logre2__C': [0.1, 1, 10],
    'logre2__penalty': ['l1', 'l2'], # Lasso y Ridge
    'logre2__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], # se prueba cada algoritmo
    'logre2__class_weight': [None, 'balanced'] # sin balanceo y con balanceo
}


In [47]:
# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Obtener las predicciones del mejor modelo encontrado
y_pred_log = grid_search.predict(X_test)

# Calcular el classification report
report_log = classification_report(y_test, y_pred_log)
print("Classification Report:")
print(report_log)


Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     58367
           1       0.50      0.08      0.14      5592

    accuracy                           0.91     63959
   macro avg       0.71      0.54      0.55     63959
weighted avg       0.88      0.91      0.88     63959



In [48]:
grid_search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('logre2',
                 LogisticRegression(C=0.1, penalty='l1', solver='saga'))])

In [49]:
grid_search.best_params_

{'logre2__C': 0.1,
 'logre2__class_weight': None,
 'logre2__penalty': 'l1',
 'logre2__solver': 'saga'}

In [52]:
from sklearn.model_selection import GridSearchCV

# Crear el pipeline con escalamiento y modelo Logistic Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selectkbest', SelectKBest()),
    ('logre2', LogisticRegression())
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'logre2__C': [0.1, 1, 10],
    'logre2__penalty': ['l1', 'l2'], # Lasso y Ridge
    'logre2__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # se prueba cada algoritmo
    'logre2__class_weight': [None, 'balanced'],  # sin balanceo y con balanceo
    'selectkbest__k': [5, 10, 15]
}

In [53]:

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='recall')
grid_search.fit(X_train, y_train)

# Obtener las predicciones del mejor modelo encontrado
y_pred2 = grid_search.predict(X_test)

In [54]:
# Calcular el classification report
report2 = classification_report(y_test, y_pred2)
print("Classification Report:")
print(report2)

# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:")
print(best_params)


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.69      0.81     58367
           1       0.19      0.77      0.31      5592

    accuracy                           0.70     63959
   macro avg       0.58      0.73      0.56     63959
weighted avg       0.90      0.70      0.76     63959

Mejores hiperparámetros:
{'logre2__C': 0.1, 'logre2__class_weight': 'balanced', 'logre2__penalty': 'l1', 'logre2__solver': 'liblinear', 'selectkbest__k': 5}


In [1]:
lr = LogisticRegression()

param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0]
}

grid_search_lr = GridSearchCV(lr, param_grid=param_grid_lr, scoring='accuracy', cv=5)
grid_search_lr.fit(X_train, y_train)
best_params_lr = grid_search_lr.best_params_


NameError: name 'LogisticRegression' is not defined

#### NAIVE BAYES

#### SVC

#### DECISSION TREE CLASSIFIER

#### RANDOM FOREST CLASSIFIER

#### GAUSSIANNB

#### XGBCCLASSIFIER

### MODELO NO SUPERVISADO

**¿Qué es el aprendizaje no supervisado?**


#### KNN CLASSIIFIER

#### PCA

### REDES NEURONALES